In [1]:
import requests
import json
import pymysql

# URL pública do JSON no GCP
public_url = "https://storage.googleapis.com/restaurante-datalake/raw/guest-checks/ERP.json"

# Fazendo a requisição GET ao link ypblico
response = requests.get(public_url)
    
# Verificando se a requisição foi bem-sucedida
if response.status_code == 200:
    data = json.loads(response.text)
    print("Dados carregados com sucesso:")
    print(data)
else:
    print(f"Erro ao acessar o arquivo: {response.status_code}")

Dados carregados com sucesso:
{'curUTC': '2024-05-05T06:06:06', 'locRef': '99 CB CB', 'guestChecks': [{'guestCheckId': 1122334455, 'chkNum': 1234, 'opnBusDt': '2024-01-01', 'opnUTC': '2024-01-01T09:09:09', 'opnLcl': '2024-01-01T06:09:09', 'clsdBusDt': '2024-01-01', 'clsdUTC': '2024-01-01T12:12:12', 'clsdLcl': '2024-01-01T09:12:12', 'lastTransUTC': '2024-01-01T12:12:12', 'lastTransLcl': '2024-01-01T09:12:12', 'lastUpdatedUTC': '2024-01-01T13:13:13', 'lastUpdatedLcl': '2024-01-01T10:13:13', 'clsdFlag': True, 'gstCnt': 1, 'subTtl': 109.9, 'nonTxblSlsTtl': None, 'chkTtl': 109.9, 'dscTtl': -10, 'payTtl': 109.9, 'balDueTtl': None, 'rvcNum': 101, 'otNum': 1, 'ocNum': None, 'tblNum': 1, 'tblName': '90', 'empNum': 55555, 'numSrvcRd': 3, 'numChkPrntd': 2, 'taxes': [{'taxNum': 28, 'txblSlsTtl': 119.9, 'taxCollTtl': 20.81, 'taxRate': 21, 'type': 3}], 'detailLines': [{'guestCheckLineItemId': 9988776655, 'rvcNum': 123, 'dtlOtNum': 1, 'dtlOcNum': None, 'lineNum': 1, 'dtlId': 1, 'detailUTC': '2024-01-

In [2]:
# Conexão com o banco de dados MySQL
connection = pymysql.connect(
    host="localhost",
    user="root",  # Altere para o seu usuário
    password="-------",  # Insira sua senha
    database="db_desafio"  # Mantenha o mesmo nome pois é o nome que o databse criado no MySQL usará
)


try:
    with connection.cursor() as cursor:
        # Inserir dados na tabela guestChecks
        for guestCheck in data["guestChecks"]:
            cursor.execute("""
                INSERT INTO guestChecks (
                    guestCheckId, chkNum, opnBusDt, opnUTC, opnLcl, clsdBusDt, clsdUTC, clsdLcl,
                    lastTransUTC, lastTransLcl, lastUpdatedUTC, lastUpdatedLcl, clsdFlag, gstCnt,
                    subTtl, nonTxblSlsTtl, chkTtl, dscTtl, payTtl, balDueTtl, rvcNum, otNum, ocNum,
                    tblNum, tblName, empNum, numSrvcRd, numChkPrntd
                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """, (
                guestCheck["guestCheckId"],
                guestCheck["chkNum"],
                guestCheck["opnBusDt"],
                guestCheck.get("opnUTC"),
                guestCheck.get("opnLcl"),
                guestCheck.get("clsdBusDt"),
                guestCheck.get("clsdUTC"),
                guestCheck.get("clsdLcl"),
                guestCheck.get("lastTransUTC"),
                guestCheck.get("lastTransLcl"),
                guestCheck.get("lastUpdatedUTC"),
                guestCheck.get("lastUpdatedLcl"),
                guestCheck.get("clsdFlag"),
                guestCheck["gstCnt"],
                guestCheck["subTtl"],
                guestCheck.get("nonTxblSlsTtl"),
                guestCheck["chkTtl"],
                guestCheck.get("dscTtl"),
                guestCheck.get("payTtl"),
                guestCheck.get("balDueTtl"),
                guestCheck.get("rvcNum"),
                guestCheck.get("otNum"),
                guestCheck.get("ocNum"),
                guestCheck.get("tblNum"),
                guestCheck.get("tblName"),
                guestCheck.get("empNum"),
                guestCheck.get("numSrvcRd"),
                guestCheck.get("numChkPrntd")
            ))

            # Inserir dados na tabela taxes
            for tax in guestCheck["taxes"]:
                cursor.execute("""
                    INSERT INTO taxes (guestCheckId, taxNum, txblSlsTtl, taxCollTtl, taxRate, type)
                    VALUES (%s, %s, %s, %s, %s, %s)
                """, (
                    guestCheck["guestCheckId"],
                    tax["taxNum"],
                    tax["txblSlsTtl"],
                    tax["taxCollTtl"],
                    tax["taxRate"],
                    tax["type"]
                ))

            # Inserir dados na tabela detailLines e suas dependências
            for detailLine in guestCheck["detailLines"]:
                cursor.execute("""
                    INSERT INTO detailLines (
                        guestCheckLineItemId, guestCheckId, dspTtl, seatNum, rvcNum, dtlOtNum,
                        dtlOcNum, lineNum, dtlId, detailUTC, detailLcl, lastUpdateUTC,
                        lastUpdateLcl, busDt, wsNum, aggTtl, aggQty, svcRndNum, chkEmpId, chkEmpNum, dspQty
                    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """, (
                    detailLine["guestCheckLineItemId"],
                    guestCheck["guestCheckId"],
                    detailLine["dspTtl"],
                    detailLine.get("seatNum"),
                    detailLine.get("rvcNum"),
                    detailLine.get("dtlOtNum"),
                    detailLine.get("dtlOcNum"),
                    detailLine.get("lineNum"),
                    detailLine.get("dtlId"),
                    detailLine.get("detailUTC"),
                    detailLine.get("detailLcl"),
                    detailLine.get("lastUpdateUTC"),
                    detailLine.get("lastUpdateLcl"),
                    detailLine.get("busDt"),
                    detailLine.get("wsNum"),
                    detailLine.get("aggTtl"),
                    detailLine.get("aggQty"),
                    detailLine.get("svcRndNum"),
                    detailLine.get("chkEmpId"),
                    detailLine.get("chkEmpNum"),
                    detailLine.get("dspQty")
                ))

                # Inserir dados na tabela menuItem
                menuItem = detailLine["menuItem"]
                cursor.execute("""
                    INSERT INTO menuItem (
                        miNum, guestCheckLineItemId, modFlag, inclTax, activeTaxes, prcLvl
                    ) VALUES (%s, %s, %s, %s, %s, %s)
                """, (
                    menuItem["miNum"],
                    detailLine["guestCheckLineItemId"],
                    menuItem["modFlag"],
                    menuItem["inclTax"],
                    json.dumps(menuItem.get("activeTaxes")),
                    menuItem.get("prcLvl")
                ))

        # Confirmar as alterações no banco de dados
        connection.commit()

except Exception as e:
    print(f"Erro ao inserir os dados: {e}")
    connection.rollback()

finally:
    connection.close()